In [1]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv("C:\\Users\\ajaco\\Desktop\\repos\\noreallyimfine\\ebird-project\\birds_and_regions.csv")

print(df.shape)

df.head()

(382014, 22)


,common_name,observ_count,country,state,county,lat,lon,observ_date,observ_year,observ_month,...,CountyName,CountyFIPS,Region,RegionName,DensityCat,DensityCatName,Population2016,Density2016_SqMi,Density2016_PFbF,DensityAggreg
0,Magnolia Warbler,2,United States,Illinois,Cook,41.775629,-87.583273,1995-08-27,1995,8,...,Cook,13075,1,01 Deep South,3,3 Rural/Semi-Rural,17167,86.093266,0.177879,100
1,Magnolia Warbler,2,United States,Illinois,Cook,41.775629,-87.583273,1995-08-27,1995,8,...,Cook,17031,8,08 Great Lakes,1,1 Urban,5203499,13472.458286,27.835658,100
2,Magnolia Warbler,2,United States,Illinois,Cook,41.775629,-87.583273,1995-08-27,1995,8,...,Cook,27031,8,08 Great Lakes,3,3 Rural/Semi-Rural,5286,11.560327,0.023885,100
3,Great Blue Heron,2,United States,Illinois,Cook,41.956746,-87.913027,1999-07-24,1999,7,...,Cook,13075,1,01 Deep South,3,3 Rural/Semi-Rural,17167,86.093266,0.177879,100
4,Great Blue Heron,2,United States,Illinois,Cook,41.956746,-87.913027,1999-07-24,1999,7,...,Cook,17031,8,08 Great Lakes,1,1 Urban,5203499,13472.458286,27.835658,100


In [4]:
df.isnull().sum()

common_name         0
observ_count        0
country             0
state               0
county              0
lat                 0
lon                 0
observ_date         0
observ_year         0
observ_month        0
season              0
State               0
CountyName          0
CountyFIPS          0
Region              0
RegionName          0
DensityCat          0
DensityCatName      0
Population2016      0
Density2016_SqMi    0
Density2016_PFbF    0
DensityAggreg       0
dtype: int64

In [5]:
df.query("common_name == 'Magnolia Warbler'")

,common_name,observ_count,country,state,county,lat,lon,observ_date,observ_year,observ_month,...,CountyName,CountyFIPS,Region,RegionName,DensityCat,DensityCatName,Population2016,Density2016_SqMi,Density2016_PFbF,DensityAggreg
0,Magnolia Warbler,2,United States,Illinois,Cook,41.775629,-87.583273,1995-08-27,1995,8,...,Cook,13075,1,01 Deep South,3,3 Rural/Semi-Rural,17167,86.093266,0.177879,100
1,Magnolia Warbler,2,United States,Illinois,Cook,41.775629,-87.583273,1995-08-27,1995,8,...,Cook,17031,8,08 Great Lakes,1,1 Urban,5203499,13472.458286,27.835658,100
2,Magnolia Warbler,2,United States,Illinois,Cook,41.775629,-87.583273,1995-08-27,1995,8,...,Cook,27031,8,08 Great Lakes,3,3 Rural/Semi-Rural,5286,11.560327,0.023885,100
129,Magnolia Warbler,1,United States,Illinois,Cook,41.850030,-87.650050,1972-09-24,1972,9,...,Cook,13075,1,01 Deep South,3,3 Rural/Semi-Rural,17167,86.093266,0.177879,100
130,Magnolia Warbler,1,United States,Illinois,Cook,41.850030,-87.650050,1972-09-24,1972,9,...,Cook,17031,8,08 Great Lakes,1,1 Urban,5203499,13472.458286,27.835658,100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375640,Magnolia Warbler,1,United States,Kentucky,Henderson,37.848805,-87.740117,1988-09-25,1988,9,...,Henderson,47077,2,02 Appohzarka,3,3 Rural/Semi-Rural,27822,100.964644,0.208605,100
375641,Magnolia Warbler,1,United States,Kentucky,Henderson,37.848805,-87.740117,1988-09-25,1988,9,...,Henderson,48213,1,01 Deep South,3,3 Rural/Semi-Rural,79901,320.880799,0.662977,100
376934,Magnolia Warbler,1,United States,Alabama,Tuscaloosa,33.223844,-87.523860,1983-05-06,1983,5,...,Tuscaloosa,1125,1,01 Deep South,3,3 Rural/Semi-Rural,206102,1347.462672,2.784014,100
378875,Magnolia Warbler,1,United States,Michigan,Eaton,42.691836,-84.629960,1968-05-22,1968,5,...,Eaton,26045,8,08 Great Lakes,3,3 Rural/Semi-Rural,109160,982.444380,2.029844,100
